In [ ]:
!pip install gradio pandas

In [ ]:
pip install --upgrade gradio

In [3]:
import gradio as gr
import pandas as pd
import tempfile

# Initialize file path
data_file_path = "/content/score_example.xlsx"

# Initialize data
data = pd.DataFrame()
user_temp_storage = {}  # Dictionary to store temporary data for each user

# Load data during initialization
def initialize_data():
    global data
    try:
        data = pd.read_excel(data_file_path)
        total_questions = len(data)
        return f"資料已成功載入！總問題數: {total_questions} 題"
    except Exception as e:
        return f"資料載入失敗！錯誤: {str(e)}"

# Query question and standard answer
def query_question_and_answer(row_number, user_id):
    global user_temp_storage
    if user_id not in user_temp_storage:
        user_temp_storage[user_id] = {
            "temp_storage": pd.DataFrame(columns=["行號", "問題", "標準答案", "模型編號", "模型答案", "評分"]),
            "current_index": 0
        }

    user_data = user_temp_storage[user_id]
    user_data["current_index"] = 0  # Reset to start from A

    if data.empty:
        return "未載入資料！請檢查資料文件是否存在。", "", ""
    try:
        row_number = int(row_number) - 1
    except ValueError:
        return "輸入錯誤！請輸入有效的行號。", "", ""

    if row_number < 0 or row_number >= len(data):
        return "行號超出範圍，請重新輸入。", "", ""

    selected_row = data.iloc[row_number]
    question = f"問題內容：\n{selected_row['Question']}"
    standard_answer = f"標準答案：\n{selected_row['Standard Answer']}"
    return question, standard_answer, f"模型答案：\n{selected_row[f'模型A']}"

# Switch model answer
def switch_model_answer(row_number, direction, user_id):
    global user_temp_storage
    if user_id not in user_temp_storage:
        return "未找到使用者記錄，請重新查詢問題。", ""

    user_data = user_temp_storage[user_id]
    current_index = user_data["current_index"]

    if data.empty:
        return "未載入資料！請檢查資料文件是否存在。", ""

    try:
        row_number = int(row_number) - 1
    except ValueError:
        return "輸入錯誤！請輸入有效的行號。", ""

    if row_number < 0 or row_number >= len(data):
        return "行號超出範圍，請重新輸入。", ""

    if direction == "next":
        current_index = (current_index + 1) % 6
    elif direction == "prev":
        current_index = (current_index - 1 + 6) % 6

    user_data["current_index"] = current_index

    model_column = f"模型{chr(65 + current_index)}"
    model_answer = data.iloc[row_number].get(model_column, "")

    if pd.isna(model_answer) or model_answer.strip() == "":
        return f"{model_column} 的答案為空，請檢查數據。", ""

    model = f"模型 {chr(65 + current_index)}"
    return f"\n{model_answer}", model

# Save score
def save_score(row_number, score, user_id):
    global user_temp_storage
    if user_id not in user_temp_storage:
        return "未找到使用者記錄，請重新查詢問題。"

    user_data = user_temp_storage[user_id]
    temp_storage = user_data["temp_storage"]
    current_index = user_data["current_index"]

    if data.empty:
        return "未載入資料！請檢查資料文件是否存在。"
    try:
        row_number = int(row_number) - 1
    except ValueError:
        return "輸入錯誤！請輸入有效的行號。"

    if row_number < 0 or row_number >= len(data):
        return "行號超出範圍，請重新輸入。"

    selected_row = data.iloc[row_number]
    model_column = f"模型{chr(65 + current_index)}"

    if model_column not in selected_row:
        return f"模型欄位 {model_column} 不存在，請檢查數據文件。"

    model_answer = selected_row.get(model_column, "")

    if pd.isna(model_answer) or model_answer.strip() == "":
        return f"行號 {row_number + 1} 的 {model_column} 答案為空，請檢查數據。"

    new_entry = {
        "行號": row_number + 1,
        "問題": selected_row["Question"],
        "標準答案": selected_row["Standard Answer"],
        "模型編號": chr(65 + current_index),  # Save the correct model index
        "模型答案": model_answer,
        "評分": score,
    }
    user_data["temp_storage"] = pd.concat([temp_storage, pd.DataFrame([new_entry])], ignore_index=True)
    return "評分已成功暫存！"

# Download scores
def download_scores(user_id):
    global user_temp_storage
    if user_id not in user_temp_storage:
        return None, "未找到使用者記錄，請重新查詢問題。"

    temp_storage = user_temp_storage[user_id]["temp_storage"]

    if temp_storage.empty:
        return None, "暫存中無評分記錄，請先暫存評分！"

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx")
    temp_storage.to_excel(temp_file.name, index=False, engine="openpyxl")
    return temp_file.name

# Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 問題查詢與評分系統")

    with gr.Row():
        user_id = gr.Textbox(label="使用者 ID", placeholder="請輸入您的使用者名稱")
        load_status = gr.Textbox(label="資料載入狀態", value=initialize_data(), interactive=False)

    with gr.Row():
        row_query = gr.Number(label="查詢行號", value=1)
        query_btn = gr.Button("查詢問題")

    with gr.Row():
        query_result = gr.Textbox(label="問題", lines=10, interactive=False)

    with gr.Row():
        standard_answer_result = gr.Textbox(label="標準答案", lines=5, interactive=False)
        model_answer_result = gr.Textbox(label="模型答案", lines=5, interactive=False)

    with gr.Row():
        score_input = gr.Number(label="評分", value=0, step=1)
        save_score_btn = gr.Button("暫存評分")
        save_score_status = gr.Textbox(label="暫存狀態", interactive=False)

    with gr.Row():
        prev_model_btn = gr.Button("上一個模型答案")
        next_model_btn = gr.Button("下一個模型答案")

    with gr.Row():
        download_btn = gr.Button("下載評分")
        download_link = gr.File(label="下載文件")

    # Event bindings
    query_btn.click(
        query_question_and_answer,
        inputs=[row_query, user_id],
        outputs=[query_result, standard_answer_result, model_answer_result],
    )
    prev_model_btn.click(
        switch_model_answer,
        inputs=[row_query, gr.State("prev"), user_id],
        outputs=[model_answer_result, gr.State()],
    )
    next_model_btn.click(
        switch_model_answer,
        inputs=[row_query, gr.State("next"), user_id],
        outputs=[model_answer_result, gr.State()],
    )
    save_score_btn.click(
        save_score,
        inputs=[row_query, score_input, user_id],
        outputs=[save_score_status],
    )
    download_btn.click(
        download_scores,
        inputs=[user_id],
        outputs=[download_link],
    )

# Launch the Gradio interface
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://228fe7acb4c50ee9ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
